# Generating snp_dist matrix for tSNE analysis - Infantis

In [13]:
# Importing libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
import io
from sklearn.decomposition import PCA
from Bio.Phylo.TreeConstruction import _Matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
pd.set_option('display.max_columns', 300)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/joaocarlosgomesneto/miniconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['matrix']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
# Function to create matrix using snp_distance produced by snp_dists for group 1


def create_matrix(fname):
    d = pd.read_csv(fname, header = 0)
    d = d.rename(columns = {'Unnamed: 0':''})
    d = d.replace(np.NaN, '')
    d1 = d
    d1_labels = d.columns
    d2_labels = pd.DataFrame(d1_labels)
    d2_labels = d1_labels.transpose()
    names = d1_labels.to_list()
    # first record is empty, remove it
    names.pop(0)
    d.to_csv('/Users/joaocarlosgomesneto/Documents/frontiers_paper_salmonella_newport_typhimurium/data/infantis/snp_matrix/d.csv', header = False, index = False)
    f1 = '/Users/joaocarlosgomesneto/Documents/frontiers_paper_salmonella_newport_typhimurium/data/infantis/snp_matrix/d.csv'
    df1 = pd.read_csv(f1, header = None)
    df2 = df1.drop([0], axis = 1)
    df3 = df2.replace(np.NaN, '')
    df4 = np.array(df3)
    df5 = np.tril(df4)
    df6 = np.array(df5).tolist()
    # extract lower triangualar matrix
    lower = []
    for i in range(0, len(df6)):
        tmp = []
        tmp = df6[i][:i]
        lower.append(tmp)
    # include diagonal to lower triangular matrix
    for i in range(0, len(lower)):
        lower[i].insert(len(lower[i]), 0)
    matrix = lower
    m = _Matrix(names, matrix)
    return m

In [15]:
# Run function to get the matrix

f1 = '/Users/joaocarlosgomesneto/Documents/frontiers_paper_salmonella_newport_typhimurium/data/infantis/input_data/distance_snp_sites_infantis.tsv'
matrix = create_matrix(f1)
snpdistinfantis = matrix

In [16]:
# Run the tSNE program

X_tsne = TSNE(learning_rate=200, n_components = 2, n_iter = 1000, random_state = 1).fit_transform(snpdistinfantis)

In [17]:
X_tsne #get the output

array([[ 25.816448 ,   4.3761706],
       [ 57.606873 ,  13.383733 ],
       [ 59.15357  ,  11.535751 ],
       ...,
       [  5.0146794, -30.71348  ],
       [ -2.0583756, -41.92949  ],
       [-37.196    , -27.932438 ]], dtype=float32)

In [18]:
# Create a dataframe with tSNE output

np.random.seed(1)
a = pd.DataFrame(X_tsne)
a.columns = ['tSNE1', 'tSNE2']
a = a.reset_index()

In [19]:
a

,index,tSNE1,tSNE2
0,0,25.816448,4.376171
1,1,57.606873,13.383733
2,2,59.153568,11.535751
3,3,-8.785485,5.999981
4,4,61.510338,23.728907
...,...,...,...
2846,2846,16.078827,-61.671326
2847,2847,15.460892,-62.199333
2848,2848,5.014679,-30.713480
2849,2849,-2.058376,-41.929489


In [20]:
# Get the genome ids

b = pd.DataFrame(snpdistinfantis.names)
b.columns = ['id']
b = b.reset_index()

In [21]:
b

,index,id
0,0,ERR3181860
1,1,SRR1002832
2,2,SRR1002834
3,3,SRR10222594
4,4,SRR10240828
...,...,...
2846,2846,SRR9984488
2847,2847,SRR9984489
2848,2848,SRR9984501
2849,2849,SRR9984504


In [22]:
# Merge all datasets

snp_infantis = pd.merge(b, a, on = 'index')
snp_infantis = snp_infantis[['id', 'tSNE1', 'tSNE2']]

In [23]:
snp_infantis

,id,tSNE1,tSNE2
0,ERR3181860,25.816448,4.376171
1,SRR1002832,57.606873,13.383733
2,SRR1002834,59.153568,11.535751
3,SRR10222594,-8.785485,5.999981
4,SRR10240828,61.510338,23.728907
...,...,...,...
2846,SRR9984488,16.078827,-61.671326
2847,SRR9984489,15.460892,-62.199333
2848,SRR9984501,5.014679,-30.713480
2849,SRR9984504,-2.058376,-41.929489


In [24]:
# Export the data 

snp_infantis.to_csv('/Users/joaocarlosgomesneto/Documents/frontiers_paper_salmonella_newport_typhimurium/data/infantis/snp_matrix/snp_infantis.csv', header = True, index = False)